In [2]:
import pandas as pd
import io
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt

import albumentations as A
from albumentations.pytorch import ToTensorV2

c:\Users\am969\anaconda3\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
import torch
import torch.nn as nn
import  torchvision.transforms.functional as TF

modelo: (`model.py`)

In [4]:
import torch
import torch.nn as nn
import  torchvision.transforms.functional as TF

# Bloque de código de cada capa de la Unet (donde se aplican las dos convoluciones, Además se aplica el btchnorm):
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
        self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size = 2, stride=2)

        # Down part of Unet:
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of Unet:
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottlenech = DoubleConv(features[-1], features[-1]*2) # Medium part
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1) #Final part


    def forward(self, x):
        skip_connections = []
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottlenech(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

def test():
    x = torch.randn((3, 1, 161, 161))
    model = UNET(in_channels=1, out_channels=1)
    preds = model(x)
    print(preds.shape)
    print(x.shape)
    assert preds.shape == x.shape

In [5]:
test()

torch.Size([3, 1, 161, 161])
torch.Size([3, 1, 161, 161])


Cargar el zip:

In [7]:
# CARGAR EL MODELO DEL ZIP (solo para guardar este código):
# prompt: Ahora dame el código con el que lo cargaría en el otro script, asignándolo a la variable model2
# import torch
# from model import UNET  # Assuming your UNET class is in a file named model.py
import zipfile

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_model_from_zip(zip_filename, device="cpu"):
  """Loads a PyTorch model from a zip file.

  Args:
    zip_filename: The name of the zip file containing the model weights.
    device: The device to load the model onto (e.g., "cuda" or "cpu").

  Returns:
    The loaded PyTorch model.
  """

  with zipfile.ZipFile(zip_filename, 'r') as zipf:
    zipf.extractall()

  model = UNET(in_channels=3, out_channels=1).to(device)
  model.load_state_dict(torch.load("model2_weights.pth", map_location=device))

  return model


# Example of how to load the model
model2 = load_model_from_zip("C:/Users/am969/Documents/DFU_Proyect/SegmentationNetworks/trained_Models/Models_Zps_Imgs/model_061124/model2.zip", device=DEVICE)

C:\Users\am969\AppData\Local\Temp\ipykernel_19104\3887677613.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model2_weights.pth", map_

In [31]:
# prompt: ahora calculame el accuracy y el coeficiente dice del modelo entrenado con ayuda de las carpetas /content/FootSegmentation_4/images-unet/IMs29Ago/test_images y /content/FootSegmentation_4/images-unet/IMs29Ago/test_masks

def calculate_metrics(test_image_dir, test_mask_dir, model, device="cuda", image_height=224, image_width=224):
  """Calculates accuracy and Dice coefficient for a trained model on a test dataset.

  Args:
    test_image_dir: Path to the directory containing test images.
    test_mask_dir: Path to the directory containing test masks.
    model: The trained PyTorch model.
    device: The device to run the model on (e.g., "cuda" or "cpu").
    image_height: The height to resize images to before feeding them to the model.
    image_width: The width to resize images to before feeding them to the model.

  Returns:
    A tuple containing the accuracy and Dice coefficient.
  """

  num_correct = 0
  num_pixels = 0
  dice_score = 0
  model.eval()

  val_transforms = A.Compose(
      [
          A.Resize(height=image_height, width=image_width),
          A.Normalize(
              mean=[0.0, 0.0, 0.0],
              std=[1.0, 1.0, 1.0],
              max_pixel_value=255.0,
          ),
          ToTensorV2(),
      ]
  )


  image_files = [f for f in os.listdir(test_image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
  for image_file in image_files:
      img_path = os.path.join(test_image_dir, image_file)
      mask_path = os.path.join(test_mask_dir, image_file)
      image = np.array(Image.open(img_path).convert("RGB"))
      mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
      mask[mask == 255.0] = 1.0

      augmentations = val_transforms(image=image, mask=mask)
      image = augmentations["image"]
      mask = augmentations["mask"]


      x = image.unsqueeze(0).to(device)
      y = torch.tensor(mask).unsqueeze(0).unsqueeze(0).to(device)


      with torch.no_grad():
          preds = torch.sigmoid(model(x))
          preds = (preds > 0.5).float()
          num_correct += (preds == y).sum()
          num_pixels += torch.numel(preds)
          dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8)

  accuracy = num_correct / num_pixels if num_pixels > 0 else 0
  dice_coefficient = dice_score / len(image_files) if len(image_files) > 0 else 0

  model.train()

  return accuracy, dice_coefficient

In [32]:
# Example usage:
# test_image_dir = "/content/FootSegmentation_4/images-unet/IMs29Ago/test_images"
# test_mask_dir = "/content/FootSegmentation_4/images-unet/IMs29Ago/test_masks"
test_image_dir = "C:/Users/am969/Documents/DFU_Proyect/data_DFU_images/Images_Gerardo/testing_12/testing-images/images/"
test_mask_dir = "C:/Users/am969/Documents/DFU_Proyect/data_DFU_images/Images_Gerardo/testing_12/testing-images/true_masks/"

accuracy, dice_coefficient = calculate_metrics(test_image_dir, test_mask_dir, model2, device=DEVICE)
print(f"Accuracy: {accuracy:.4f}")
print(f"Dice Coefficient: {dice_coefficient:.4f}")

C:\Users\am969\AppData\Local\Temp\ipykernel_19104\2375844145.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(mask).unsqueeze(0).unsqueeze(0).to(device)


Accuracy: 0.9453
Dice Coefficient: 0.5446


Recortar las imágenes:

In [33]:
# prompt: Dame una función que tome como argumentos: el path de una carpeta de imágenes, así como el de una carpeta con sus respectivas máscaras binarias y que me cree otras dos carpetas (una de imágenes y otra de máscaras) de las imágnes y máscaras recortadas en un 5% del contorno.

# import os
# from PIL import Image
# import numpy as np

def crop_images_and_masks(image_dir, mask_dir, output_image_dir, output_mask_dir, crop_percentage=0.05):
    """
    Crops images and their corresponding masks by a given percentage.

    Args:
        image_dir: Path to the directory containing images.
        mask_dir: Path to the directory containing masks.
        output_image_dir: Path to the output directory for cropped images.
        output_mask_dir: Path to the output directory for cropped masks.
        crop_percentage: The percentage to crop from each edge (default: 5%).
    """

    if not os.path.exists(output_image_dir):
        os.makedirs(output_image_dir)
    if not os.path.exists(output_mask_dir):
        os.makedirs(output_mask_dir)

    for filename in os.listdir(image_dir):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_dir, filename)
            mask_path = os.path.join(mask_dir, filename)  # Assuming same filenames

            try:
                image = Image.open(image_path)
                mask = Image.open(mask_path)

                # Get image dimensions
                width, height = image.size

                # Calculate cropping amounts
                crop_width = int(width * crop_percentage)
                crop_height = int(height * crop_percentage)

                # Crop the image and mask
                cropped_image = image.crop((crop_width, crop_height, width - crop_width, height - crop_height))
                cropped_mask = mask.crop((crop_width, crop_height, width - crop_width, height - crop_height))

                # Save the cropped image and mask
                cropped_image_path = os.path.join(output_image_dir, filename)
                cropped_mask_path = os.path.join(output_mask_dir, filename)
                cropped_image.save(cropped_image_path)
                cropped_mask.save(cropped_mask_path)
            
            except FileNotFoundError:
                print(f"Mask file not found for {filename}. Skipping.")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

In [37]:

# Example usage:
image_folder = "C:/Users/am969/Documents/DFU_Proyect/data_DFU_images/Images_Gerardo/testing_12/testing-images/images/"  # Replace with your image folder path
mask_folder = "C:/Users/am969/Documents/DFU_Proyect/data_DFU_images/Images_Gerardo/testing_12/testing-images/true_masks/"      # Replace with your mask folder path
output_image_folder = "C:/Users/am969/Documents/DFU_Proyect/data_DFU_images/Images_Gerardo/testing_12/testing-images/croped_images"  # Output folder for cropped images
output_mask_folder = "C:/Users/am969/Documents/DFU_Proyect/data_DFU_images/Images_Gerardo/testing_12/testing-images/croped_masks"   # Output folder for cropped masks

crop_images_and_masks(image_folder, mask_folder, output_image_folder, output_mask_folder, crop_percentage=0.01)

Probamos con las imágenes recortadas:

In [38]:
test_image_dir = "C:/Users/am969/Documents/DFU_Proyect/data_DFU_images/Images_Gerardo/testing_12/testing-images/croped_images/"
test_mask_dir = "C:/Users/am969/Documents/DFU_Proyect/data_DFU_images/Images_Gerardo/testing_12/testing-images/croped_masks/"

accuracy, dice_coefficient = calculate_metrics(test_image_dir, test_mask_dir, model2, device=DEVICE)
print(f"Accuracy: {accuracy:.4f}")
print(f"Dice Coefficient: {dice_coefficient:.4f}")

C:\Users\am969\AppData\Local\Temp\ipykernel_19104\2375844145.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(mask).unsqueeze(0).unsqueeze(0).to(device)


Accuracy: 0.9385
Dice Coefficient: 0.5422
